## Logistic Regression

Breast Cancer data from [the UCI repository](http://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29) contains records corresponding to 
cases of observed tumors.   There are a number of observations for each and a categorisation in the `class` column: 2 for benign (good), 4 for malignant (bad).  Your task is to build a logistic regression model to classify these cases. 

The data is provided as a CSV file.  There are a small number of cases where no value is available, these are indicated in the data with `?`. I have used the `na_values` keyword for `read_csv` to have these interpreted as `NaN` (Not a Number).  Your first task is to decide what to do with these rows. You could just drop these rows or you could [impute them from the other data](http://scikit-learn.org/stable/modules/preprocessing.html#imputation-of-missing-values).

You then need to follow the procedure outlined in the lecture for generating a train/test set, building and evaluating a model. Your goal is to build the best model possible over this data.   Your first step should be to build a logistic regression model using all of the features that are available.
  

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.feature_selection import RFE

In [3]:
bcancer = pd.read_csv("data/breast-cancer-wisconsin.csv", na_values = "?")
bcancer.head()

,sample_code_number,clump_thickness,uniformity_cell_size,uniformity_cell_shape,marginal_adhesion,single_epithelial_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses,class
0,1000025,5,1,1,1,2,1.0,3,1,1,2
1,1002945,5,4,4,5,7,10.0,3,2,1,2
2,1015425,3,1,1,1,2,2.0,3,1,1,2
3,1016277,6,8,8,1,3,4.0,3,7,1,2
4,1017023,4,1,1,3,2,1.0,3,1,1,2


In [9]:
# Examine the data, look at the statistical summary etc.
bcancer.describe()

,sample_code_number,clump_thickness,uniformity_cell_size,uniformity_cell_shape,marginal_adhesion,single_epithelial_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses,class
count,6.990000e+02,699.000000,699.000000,699.000000,699.000000,699.000000,683.000000,699.000000,699.000000,699.000000,699.000000
mean,1.071704e+06,4.417740,3.134478,3.207439,2.806867,3.216023,3.544656,3.437768,2.866953,1.589413,2.689557
std,6.170957e+05,2.815741,3.051459,2.971913,2.855379,2.214300,3.643857,2.438364,3.053634,1.715078,0.951273
min,6.163400e+04,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000
25%,8.706885e+05,2.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,2.000000
50%,1.171710e+06,4.000000,1.000000,1.000000,1.000000,2.000000,1.000000,3.000000,1.000000,1.000000,2.000000
75%,1.238298e+06,6.000000,5.000000,5.000000,4.000000,4.000000,6.000000,5.000000,4.000000,1.000000,4.000000
max,1.345435e+07,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,4.000000


In [8]:
bcancer.shape

(699, 11)

In [13]:
# deal with the NaN values in the data
clean_bcancer = bcancer.dropna(how = 'any')
clean_bcancer.shape

(683, 11)

In [15]:
# Build your first model - defining training and test data sets then use Logistic Regression to build a model
train, test = train_test_split(clean_bcancer, test_size = 0.2, random_state = 7)
print('Dimension of training dataset: ', train.shape)
print('Dimension of testing dataset: ', test.shape)

Dimension of training dataset:  (546, 11)
Dimension of testing dataset:  (137, 11)


In [16]:
train.columns

Index(['sample_code_number', 'clump_thickness', 'uniformity_cell_size',
       'uniformity_cell_shape', 'marginal_adhesion',
       'single_epithelial_cell_size', 'bare_nuclei', 'bland_chromatin',
       'normal_nucleoli', 'mitoses', 'class'],
      dtype='object')

In [18]:
X_cols = ['clump_thickness', 'uniformity_cell_size', 'uniformity_cell_shape', 'marginal_adhesion', 'single_epithelial_cell_size', 'bare_nuclei', 'bland_chromatin', 'normal_nucleoli', 'mitoses']
y_col = 'class'
X_train = train[X_cols]
y_train = train[y_col]
X_test = test[X_cols]
y_test = test[y_col]
print('Dimensions of X_train dataset: ', X_train.shape)
print('Dimensions of y_train dataset: ', y_train.shape)
print('Dimensions of X_test dataset: ', X_test.shape)
print('Dimensions of y_test dataset: ', y_test.shape)

Dimensions of X_train dataset:  (546, 9)
Dimensions of y_train dataset:  (546,)
Dimensions of X_test dataset:  (137, 9)
Dimensions of y_test dataset:  (137,)


In [20]:
model = LogisticRegression()
model.fit(X_train, y_train)

/Users/jvlpz/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

### Evaluation

To evaluate a classification model we want to look at how many cases were correctly classified and how many
were in error.  In this case we have two outcomes - benign and malignant.   SKlearn has some useful tools, the 
[accuracy_score]() function gives a score from 0-1 for the proportion correct.  The 
[confusion_matrix](http://scikit-learn.org/stable/modules/model_evaluation.html#confusion-matrix) function 
shows how many were classified correctly and what errors were made.  Use these to summarise the performance of 
your model (these functions have already been imported above).

In [35]:
y_test.value_counts()

2    89
4    48
Name: class, dtype: int64

In [39]:
# Evaluate the model on training dataset
train_pred = model.predict(X_train)
print('Train accuracy: ', accuracy_score(y_train, train_pred))
print('Confusion matrix')
print(confusion_matrix(y_train, train_pred))

Train accuracy:  0.9706959706959707
Confusion matrix
[[349   6]
 [ 10 181]]


In [40]:
# Evaluate the model on testing datase
test_pred = model.predict(X_test)
print('Test accuracy: ', accuracy_score(y_test, test_pred))
print('Confusion matrix')
print(confusion_matrix(y_test, test_pred))

Test accuracy:  0.9708029197080292
Confusion matrix
[[86  3]
 [ 1 47]]


### Feature Selection

Since you have many features available, one part of building the best model will be to select which features to use as input to the classifier. Your initial model used all of the features but it is possible that a better model can 
be built by leaving some of them out.   Test this by building a few models with subsets of the features - how do your models perform? 

This process can be automated.  The [sklearn RFE function](http://scikit-learn.org/stable/modules/feature_selection.html#recursive-feature-elimination) implements __Recursive Feature Estimation__ which removes 
features one by one, evaluating the model each time and selecting the best model for a target number of features.  Use RFE to select features for a model with 3, 4 and 5 features - can you build a model that is as good or better than your initial model?

In [66]:
model = LogisticRegression()
rfe = RFE(estimator = model, n_features_to_select = 5, step = 1)
rfe.fit(X_train, y_train)

/Users/jvlpz/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jvlpz/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jvlpz/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jvlpz/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jvlpz/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs

RFE(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                 fit_intercept=True, intercept_scaling=1,
                                 l1_ratio=None, max_iter=100,
                                 multi_class='warn', n_jobs=None, penalty='l2',
                                 random_state=None, solver='warn', tol=0.0001,
                                 verbose=0, warm_start=False),
    n_features_to_select=5, step=1, verbose=0)

In [67]:
rfe_train_pred = rfe.predict(X_train)

print('Train accuracy with 5 features: ', accuracy_score(y_train, rfe_train_pred))
print('Confusion matrix')
print(confusion_matrix(y_train, rfe_train_pred))

Train accuracy:  0.9706959706959707
Confusion matrix
[[349   6]
 [ 10 181]]


In [78]:
rfe_test_pred = rfe.predict(X_test)

print('Test accuracy with 5 features: ', accuracy_score(y_test, rfe_test_pred))
print('Confusion matrix')
print(confusion_matrix(y_test, rfe_test_pred))

Test accuracy:  0.9635036496350365
Confusion matrix
[[86  3]
 [ 2 46]]


In [69]:
features_bool = np.array(rfe.support_)
features = np.array(X_train.columns)
result = features[features_bool]
print(result)

['clump_thickness' 'uniformity_cell_size' 'marginal_adhesion'
 'bare_nuclei' 'normal_nucleoli']


In [86]:
model = LogisticRegression()
rfe = RFE(estimator = model, n_features_to_select = 4, step = 1)
rfe.fit(X_train, y_train)

/Users/jvlpz/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jvlpz/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jvlpz/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jvlpz/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jvlpz/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs

RFE(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                 fit_intercept=True, intercept_scaling=1,
                                 l1_ratio=None, max_iter=100,
                                 multi_class='warn', n_jobs=None, penalty='l2',
                                 random_state=None, solver='warn', tol=0.0001,
                                 verbose=0, warm_start=False),
    n_features_to_select=4, step=1, verbose=0)

In [87]:
rfe_train_pred = rfe.predict(X_train)

print('Train accuracy with 4 features: ', accuracy_score(y_train, rfe_train_pred))
print('Confusion matrix')
print(confusion_matrix(y_train, rfe_train_pred))

Train accuracy:  0.9688644688644689
Confusion matrix
[[349   6]
 [ 11 180]]


In [88]:
rfe_test_pred = rfe.predict(X_test)

print('Test accuracy with 4 features: ', accuracy_score(y_test, rfe_test_pred))
print('Confusion matrix')
print(confusion_matrix(y_test, rfe_test_pred))

Test accuracy:  0.9635036496350365
Confusion matrix
[[86  3]
 [ 2 46]]


In [89]:
features_bool = np.array(rfe.support_)
features = np.array(X_train.columns)
result = features[features_bool]
print(result)

['clump_thickness' 'uniformity_cell_size' 'marginal_adhesion'
 'bare_nuclei']


In [94]:
model = LogisticRegression()
rfe = RFE(estimator = model, n_features_to_select = 3, step = 1)
rfe.fit(X_train, y_train)

/Users/jvlpz/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jvlpz/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jvlpz/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jvlpz/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jvlpz/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs

RFE(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                 fit_intercept=True, intercept_scaling=1,
                                 l1_ratio=None, max_iter=100,
                                 multi_class='warn', n_jobs=None, penalty='l2',
                                 random_state=None, solver='warn', tol=0.0001,
                                 verbose=0, warm_start=False),
    n_features_to_select=3, step=1, verbose=0)

In [91]:
rfe_train_pred = rfe.predict(X_train)

print('Train accuracy with 3 features: ', accuracy_score(y_train, rfe_train_pred))
print('Confusion matrix')
print(confusion_matrix(y_train, rfe_train_pred))

Train accuracy with 3 features:  0.9633699633699634
Confusion matrix
[[348   7]
 [ 13 178]]


In [92]:
rfe_test_pred = rfe.predict(X_test)

print('Test accuracy with 3 features: ', accuracy_score(y_test, rfe_test_pred))
print('Confusion matrix')
print(confusion_matrix(y_test, rfe_test_pred))

Test accuracy with 3 features:  0.9635036496350365
Confusion matrix
[[86  3]
 [ 2 46]]


In [93]:
features_bool = np.array(rfe.support_)
features = np.array(X_train.columns)
result = features[features_bool]
print(result)

['clump_thickness' 'uniformity_cell_size' 'bare_nuclei']


In [95]:
model = LogisticRegression()
rfe = RFE(estimator = model, n_features_to_select = 2, step = 1)
rfe.fit(X_train, y_train)

/Users/jvlpz/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jvlpz/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jvlpz/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jvlpz/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jvlpz/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs

RFE(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                 fit_intercept=True, intercept_scaling=1,
                                 l1_ratio=None, max_iter=100,
                                 multi_class='warn', n_jobs=None, penalty='l2',
                                 random_state=None, solver='warn', tol=0.0001,
                                 verbose=0, warm_start=False),
    n_features_to_select=2, step=1, verbose=0)

In [97]:
rfe_test_pred = rfe.predict(X_test)

print('Test accuracy with 2 features: ', accuracy_score(y_test, rfe_test_pred))
print('Confusion matrix')
print(confusion_matrix(y_test, rfe_test_pred))

Test accuracy with 2 features:  0.9708029197080292
Confusion matrix
[[86  3]
 [ 1 47]]


## Conclusion

Write a brief conclusion to your experiment.  You might comment on the proportion of __false positive__ and __false negative__ classifications your model makes.  How useful would this model be in a clinical diagnostic setting? 

The model has a high accuracy score in predicting category of cancer. It predicted 97% of the training and testing dataset correctly.